In [ ]:
%%javascript
IPython.notebook.clear_all_output();

<IPython.core.display.Javascript object>

# Import Dataset

In [2]:
import csv
import pandas as pd
import numpy as np

In [ ]:
from datetime import datetime
import time
def datamanipulation(participant):
    ptp = participant

    path_data = "../Experiment Data/" + ptp +"/"
    path_analysis = "../Experiment Analysis/" + ptp + "/"

    responsData = pd.read_csv(path_data + "Responses.csv")
    experimentData = pd.read_csv(path_data + "Ref_Raw.csv")
#     display(responsData)
#     display(experimentData)
#     display(responsData.Start_Time[0:1])
    start_time = pd.to_datetime(responsData.WarningStartTime[0:1],format='%H:%M:%S').dt.time
    end_time = pd.to_datetime(responsData.ResponseTime[0:1],format='%H:%M:%S').dt.time
    #display(start_time)
    #display(end_time)
    clock_time = pd.to_datetime(experimentData["System_Time"],format='%H:%M:%S').dt.time
    #display(clock_time)
    #columns = ['ApplicationName','WarningName','Id','Response','ApplicationId','ParticipantId','ResponseTime','WarningId','WarningStartTime','MoreInfoClicked','MoreInfoTime','Start_Time','End_Time','Time_Spent']
    frame = []
    frameAvg = []
    frameAvgRest = []

    framefpr = []
    framefprRest = []
    for index, row in responsData.iterrows():
        ApplicationName,WarningName,Id,Response,ApplicationId,ParticipantId,ResponseTime,WarningId,WarningStartTime,MoreInfoClicked,MoreInfoTime,Time_Spent= row

        #percentage of frequecny
        pfrDIS = 0 
        pfrSO = 0
        pfrLENG = 0
        pfrHENG = 0

        #display(ApplicationName)
        #display(responsData.Start_Time[index])
        start_time = pd.to_datetime(responsData.WarningStartTime[index:index+1],format='%H:%M:%S').dt.time
        end_time = pd.to_datetime(responsData.ResponseTime[index:index+1],format='%H:%M:%S').dt.time

        #rest time 5sec
        rest_start_time = end_time
        rest_end_time = (pd.to_datetime(responsData.ResponseTime[index:index+1],format='%H:%M:%S') + pd.to_timedelta(5, unit='s')).dt.time

        mask = (clock_time < end_time[index]) & (clock_time >= start_time[index])


        mask2 = (clock_time >= rest_start_time[index]) & (clock_time < rest_end_time[index]) 

        data = experimentData[mask]
        data["AppName"] = ApplicationName
        data["WarnName"] = WarningName
        data["AppId"] = ApplicationId
        data["WarnId"] = WarningId
        data["TimeSpent"] = Time_Spent
        data["PId"] = ParticipantId
        count = 0
        for i,row2 in data.iterrows():

            so = row2["ProbSleepOnset"]
            dis = row2["ProbDistraction"]
            leng = row2["ProbLowEng"]
            heng = row2["ProbHighEng"]
            max_value = max(so,dis,leng,heng)

            if(so == max_value):
                pfrSO = pfrSO + 1
            elif(dis == max_value):
                pfrDIS = pfrDIS + 1
            elif(leng == max_value):
                pfrLENG = pfrLENG + 1
            elif(heng==max_value):
                pfrHENG = pfrHENG + 1

            count = count +1

        try:
            a["pfrSO"] = pfrSO/count
            data["pfrDIS"] = pfrDIS/count
            data["pfrLENG"] = pfrLENG/count
            data["pfrHENG"] = pfrHENG/count
        except ZeroDivisionError:
            continue

        dataRest = experimentData[mask2]
        dataRest["WarnName"] = "Rest " + WarningName
        dataRest["Rest"] = index+1
        dataRest["AppName"] = ApplicationName
        dataRest["AppId"] = ApplicationId
        dataRest["WarnId"] = WarningId
        dataRest["TimeSpent"] = "5"
        dataRest["PId"] = ParticipantId

        count = 0
        #percentage of frequecny
        pfrDIS = 0 
        pfrSO = 0
        pfrLENG = 0
        pfrHENG = 0
        for i,row3 in dataRest.iterrows():

            so = row3["ProbSleepOnset"]
            dis = row3["ProbDistraction"]
            leng = row3["ProbLowEng"]
            heng = row3["ProbHighEng"]
            max_value = max(so,dis,leng,heng)

            if(so == max_value):
                pfrSO = pfrSO + 1
            elif(dis == max_value):
                pfrDIS = pfrDIS + 1
            elif(leng == max_value):
                pfrLENG = pfrLENG + 1
            elif(heng==max_value):
                pfrHENG = pfrHENG + 1

            count = count +1

        try:
            dataRest["pfrSO"] = pfrSO/count
            dataRest["pfrDIS"] = pfrDIS/count
            dataRest["pfrLENG"] = pfrLENG/count
            dataRest["pfrHENG"] = pfrHENG/count
        except ZeroDivisionError:
            continue


        pfrData = data[['AppName','WarnName','TimeSpent','pfrSO','pfrDIS','pfrLENG','pfrHENG']]
        pfrDataRest = dataRest[['AppName','WarnName','TimeSpent','pfrSO','pfrDIS','pfrLENG','pfrHENG','PId']]

        data.loc['Avg'] = data[['ProbSleepOnset','ProbDistraction','ProbLowEng',
                                  'ProbHighEng','CogState','ProbFBDSWorkload','ProbBDSWorkload','ProbAveWorkload',"AppId","WarnId","TimeSpent","PId"
                                               ]].mean(axis=0,numeric_only=True).round(2)
        dataRest.loc['AvgRest'] = dataRest[['ProbSleepOnset','ProbDistraction','ProbLowEng',
                                  'ProbHighEng','CogState','ProbFBDSWorkload','ProbBDSWorkload','ProbAveWorkload',"AppId","WarnId","TimeSpent","PId"
                                               ]].mean(axis=0,numeric_only=True).round(2)


        if not (data.empty or dataRest.empty):
            frame.append(data)
            frame.append(dataRest)
            frameAvg.append(data.tail(1))
            frameAvgRest.append(dataRest.tail(1))
            framefpr.append(pfrData.tail(1))
            framefprRest.append(pfrDataRest.tail(1))

    #result = pd.concat(frame)
    #result2 = pd.concat(frameAvg)
    result3 = pd.concat(framefpr)
    #result4 = pd.concat(frameAvgRest)
    result5 = pd.concat(framefprRest)
    return result3,result5
    #display(result)
    result.to_csv(path_analysis+"TrialandRest.csv", index = False)
    result2.to_csv(path_analysis+"TrialAvg.csv", index = False)
    result3.to_csv(path_analysis+"CogFreqWarning.csv",index = False)
    result4.to_csv(path_analysis+"RestAvg.csv", index = False)
    result5.to_csv(path_analysis+"CogFreqWarning_Rest.csv",index = False)

    print("Done!")

In [4]:
############################# FAKE FACE ########################################################
from datetime import datetime
import time
ptp = "1000_2"

path_data = "../Data/Experiment Data/" + ptp +"/"
path_analysis = "../Data/Experiment Analysis/" + ptp + "/"

responsData = pd.read_csv(path_data + "Responses.csv")
experimentData = pd.read_csv(path_data + "Classification.csv")
#display(responsData)
#display(experimentData)
#display(responsData.StartTime[0:1])
start_time = pd.to_datetime(responsData.StartTime[0:1],format='%H:%M:%S').dt.time
end_time = pd.to_datetime(responsData.ResponseTime[0:1],format='%H:%M:%S').dt.time
#display(start_time)
#display(end_time)
clock_time = pd.to_datetime(experimentData["System_Time"],format='%H:%M:%S').dt.time
#display(clock_time)
frame = []
frameAvg = []
frameAvgRest = []

framefpr = []
#framefprRest = []
for index, row in responsData.iterrows():
    Id,CelebrityName,VideoType,Response,ParticipantId,ResponseTime,VideoId,StartTime,FamId,SeenBefore,TimeSpent = row

    #percentage of frequecny
    pfrDIS = 0 
    pfrSO = 0
    pfrLENG = 0
    pfrHENG = 0

    #display(ApplicationName)
    #display(responsData.Start_Time[index])
    start_time = pd.to_datetime(responsData.StartTime[index:index+1],format='%H:%M:%S').dt.time
    end_time = pd.to_datetime(responsData.ResponseTime[index:index+1],format='%H:%M:%S').dt.time

    mask = (clock_time < end_time[index]) & (clock_time >= start_time[index]) 

    data = experimentData[mask]    
    data["ResponseId"] = Id
    data["CelebrityName"] = CelebrityName
    data["VideoType"] = VideoType
    data["Response"] = Response
    data["ParticipantId"] = ParticipantId
    data["ResponseTime"] = ResponseTime
    data["VideoId"] = VideoId
    data["StartTime"] = StartTime
    data["FamiliarizationId"] = FamId
    data["TimeSpent"] = TimeSpent
    #data["SeenBefore"] == seenBefore
    count = 0
    #display(data)
    for i,row2 in data.iterrows():

        so = row2["ProbSleepOnset"]
        dis = row2["ProbDistraction"]
        leng = row2["ProbLowEng"]
        heng = row2["ProbHighEng"]
        max_value = max(so,dis,leng,heng)

        if(so == max_value):
            pfrSO = pfrSO + 1
        elif(dis == max_value):
            pfrDIS = pfrDIS + 1
        elif(leng == max_value):
            pfrLENG = pfrLENG + 1
        elif(heng==max_value):
            pfrHENG = pfrHENG + 1

        count = count +1

    try:
        data["pfrSO"] = pfrSO/count
        data["pfrDIS"] = pfrDIS/count
        data["pfrLENG"] = pfrLENG/count
        data["pfrHENG"] = pfrHENG/count
    except ZeroDivisionError:
        continue


    pfrData = data[['CelebrityName','VideoType','TimeSpent','pfrSO','pfrDIS','pfrLENG','pfrHENG']]    

    data.loc['Avg'] = data[['ProbSleepOnset','ProbDistraction','ProbLowEng',
                              'ProbHighEng','CogState','ProbFBDSWorkload','ProbBDSWorkload','ProbAveWorkload',"VideoId","TimeSpent","ParticipantId"
                                           ]].mean(axis=0,numeric_only=True).round(2)
    


    if not (data.empty):
        frame.append(data)        
        frameAvg.append(data.tail(1))
        framefpr.append(pfrData.tail(1))

result = pd.concat(frame)
result2 = pd.concat(frameAvg)
result3 = pd.concat(framefpr)

display(result)
#result.to_csv(path_analysis+"TrialandRest.csv", index = False)
#result2.to_csv(path_analysis+"TrialAvg.csv", index = False)
#result3.to_csv(path_analysis+"CogFreqWarning.csv",index = False)

    
    
    
print("Done!")

C:\Users\cagri\AppData\Local\Temp/ipykernel_28104/3411569583.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["ResponseId"] = Id
C:\Users\cagri\AppData\Local\Temp/ipykernel_28104/3411569583.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["CelebrityName"] = CelebrityName
C:\Users\cagri\AppData\Local\Temp/ipykernel_28104/3411569583.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

,SessionNum,Elapsed Time,Clock Time,ProbSleepOnset,ProbDistraction,ProbLowEng,ProbHighEng,CogState,ProbFBDSWorkload,ProbBDSWorkload,...,ParticipantId,ResponseTime,VideoId,StartTime,FamiliarizationId,TimeSpent,pfrSO,pfrDIS,pfrLENG,pfrHENG
454,100002000.0,00:07:34:000,18:04:42:522,1.375590e-04,0.104571,0.562248,0.333043,0.60,0.549365,0.448191,...,59.0,18:05:01,57.0,18:04:42,0:00,0:00:19,0.0,0.210526,0.315789,0.473684
455,100002000.0,00:07:35:000,18:04:43:507,4.250000e-06,0.696297,0.184836,0.118863,0.30,0.236696,0.319682,...,59.0,18:05:01,57.0,18:04:42,0:00,0:00:19,0.0,0.210526,0.315789,0.473684
456,100002000.0,00:07:36:000,18:04:44:506,8.380000e-08,0.543693,0.448821,0.007486,0.30,0.568674,0.521508,...,59.0,18:05:01,57.0,18:04:42,0:00,0:00:19,0.0,0.210526,0.315789,0.473684
457,100002000.0,00:07:37:000,18:04:45:507,1.100000e-11,0.051369,0.549522,0.399110,0.60,0.719237,0.542185,...,59.0,18:05:01,57.0,18:04:42,0:00,0:00:19,0.0,0.210526,0.315789,0.473684
458,100002000.0,00:07:38:000,18:04:46:497,3.000000e-12,0.004598,0.386409,0.608993,0.90,0.556865,0.373265,...,59.0,18:05:01,57.0,18:04:42,0:00,0:00:19,0.0,0.210526,0.315789,0.473684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1830,100002000.0,00:30:30:000,18:27:38:450,3.880000e-07,0.003561,0.498835,0.497604,0.60,0.447761,0.519466,...,59.0,18:27:42,265.0,18:27:34,0:00,0:00:08,0.0,0.125000,0.500000,0.375000
1831,100002000.0,00:30:31:000,18:27:39:447,1.640000e-09,0.028684,0.454819,0.516496,0.90,0.762100,0.630903,...,59.0,18:27:42,265.0,18:27:34,0:00,0:00:08,0.0,0.125000,0.500000,0.375000
1832,100002000.0,00:30:32:000,18:27:40:449,1.810000e-05,0.141567,0.335281,0.523134,0.90,0.868374,0.784751,...,59.0,18:27:42,265.0,18:27:34,0:00,0:00:08,0.0,0.125000,0.500000,0.375000
1833,100002000.0,00:30:33:000,18:27:41:442,1.190000e-07,0.888935,0.070810,0.040254,0.30,0.797996,0.656195,...,59.0,18:27:42,265.0,18:27:34,0:00,0:00:08,0.0,0.125000,0.500000,0.375000


Done!


In [ ]:
from datetime import datetime
import time
def datamanipulation2(participant):
    ptp = participant

    path_data = "../Experiment Data/" + ptp +"/"
    path_analysis = "../Experiment Analysis/" + ptp + "/"

    responsData = pd.read_csv(path_data + "Responses.csv")
    experimentData = pd.read_csv(path_data + "Classification.csv")
#     display(responsData)
#     display(experimentData)
#     display(responsData.Start_Time[0:1])
    start_time = pd.to_datetime(responsData.WarningStartTime[0:1],format='%H:%M:%S').dt.time
    end_time = pd.to_datetime(responsData.ResponseTime[0:1],format='%H:%M:%S').dt.time
    #display(start_time)
    #display(end_time)
    clock_time = pd.to_datetime(experimentData["System_Time"],format='%H:%M:%S').dt.time
    #display(clock_time)
    #columns = ['ApplicationName','WarningName','Id','Response','ApplicationId','ParticipantId','ResponseTime','WarningId','WarningStartTime','MoreInfoClicked','MoreInfoTime','Start_Time','End_Time','Time_Spent']
    frame = []
    frameAvg = []
    frameAvgRest = []

    framefpr = []
    framefprRest = []
    for index, row in responsData.iterrows():
        ApplicationName,WarningName,Id,Response,ApplicationId,ParticipantId,ResponseTime,WarningId,WarningStartTime,MoreInfoClicked,MoreInfoTime,Time_Spent= row

        #percentage of frequecny
        pfrDIS = 0 
        pfrSO = 0
        pfrLENG = 0
        pfrHENG = 0

        #display(ApplicationName)
        #display(responsData.Start_Time[index])
        start_time = pd.to_datetime(responsData.WarningStartTime[index:index+1],format='%H:%M:%S').dt.time
        end_time = pd.to_datetime(responsData.ResponseTime[index:index+1],format='%H:%M:%S').dt.time

        #rest time 5sec
        rest_start_time = (pd.to_datetime(responsData.ResponseTime[index:index+1],format='%H:%M:%S') + pd.to_timedelta(1,unit='s')).dt.time
        rest_end_time = (pd.to_datetime(responsData.ResponseTime[index:index+1],format='%H:%M:%S') + pd.to_timedelta(4, unit='s')).dt.time

        #mask = (clock_time < end_time[index]) & (clock_time >= start_time[index])


        mask2 = (clock_time >= rest_start_time[index]) & (clock_time < rest_end_time[index]) 

#         data = experimentData[mask]
#         data["AppName"] = ApplicationName
#         data["WarnName"] = WarningName
#         data["AppId"] = ApplicationId
#         data["WarnId"] = WarningId
#         data["TimeSpent"] = Time_Spent
#         data["PId"] = ParticipantId
#         data["Id"] = Id
#         count = 0
#         for i,row2 in data.iterrows():

#             so = row2["ProbSleepOnset"]
#             dis = row2["ProbDistraction"]
#             leng = row2["ProbLowEng"]
#             heng = row2["ProbHighEng"]
#             max_value = max(so,dis,leng,heng)

#             if(so == max_value) & (so != -99999):
#                 pfrSO = pfrSO + 1
#                 count = count +1
#             elif(dis == max_value) & (dis != -99999):
#                 pfrDIS = pfrDIS + 1
#                 count = count +1
#             elif(leng == max_value) & (leng != -99999):
#                 pfrLENG = pfrLENG + 1
#                 count = count +1
#             elif(heng==max_value) & (heng != -99999):
#                 pfrHENG = pfrHENG + 1
#                 count = count +1

            

#         try:
#             data["pfrSO"] = pfrSO/count
#             data["pfrDIS"] = pfrDIS/count
#             data["pfrLENG"] = pfrLENG/count
#             data["pfrHENG"] = pfrHENG/count
#         except ZeroDivisionError:
#             continue

        dataRest = experimentData[mask2]
        #display(dataRest)
        dataRest["WarnName"] = WarningName
        dataRest["Rest"] = index+1
        dataRest["AppName"] = ApplicationName
        dataRest["AppId"] = ApplicationId
        dataRest["WarnId"] = WarningId
        dataRest["TimeSpent"] = "5"
        dataRest["PId"] = ParticipantId

        count = 0
        #percentage of frequecny
        pfrDIS = 0 
        pfrSO = 0
        pfrLENG = 0
        pfrHENG = 0
        for i,row3 in dataRest.iterrows():

            so = row3["ProbSleepOnset"]
            dis = row3["ProbDistraction"]
            leng = row3["ProbLowEng"]
            heng = row3["ProbHighEng"]
            max_value = max(so,dis,leng,heng)

            if(so == max_value) & (so != -99999):
                pfrSO = pfrSO + 1
            elif(dis == max_value )& (dis != -99999):
                pfrDIS = pfrDIS + 1
            elif(leng == max_value )& (leng != -99999):
                pfrLENG = pfrLENG + 1
            elif(heng==max_value) & (leng != -99999):
                pfrHENG = pfrHENG + 1

            count = count +1

        try:
            dataRest["pfrSO"] = pfrSO/count
            dataRest["pfrDIS"] = pfrDIS/count
            dataRest["pfrLENG"] = pfrLENG/count
            dataRest["pfrHENG"] = pfrHENG/count
        except ZeroDivisionError:
            continue


        #pfrData = data[['Id','AppName','WarnName','TimeSpent','pfrSO','pfrDIS','pfrLENG','pfrHENG','PId']]
        pfrDataRest = dataRest[['AppName','WarnName','TimeSpent','pfrSO','pfrDIS','pfrLENG','pfrHENG','PId']]

#         data.loc['Avg'] = data[['ProbSleepOnset','ProbDistraction','ProbLowEng',
#                                   'ProbHighEng','CogState','ProbFBDSWorkload','ProbBDSWorkload','ProbAveWorkload',"AppId","WarnId","TimeSpent","PId"
#                                                ]].mean(axis=0,numeric_only=True).round(2)
#         dataRest.loc['AvgRest'] = dataRest[['AppName','WarnName','TimeSpent','pfrSO','pfrDIS','pfrLENG','pfrHENG','PId'
#                                                ]].mean(axis=0,numeric_only=True).round(3)


        if not (dataRest.empty):
            #frame.append(data)
            #frame.append(dataRest)
            #frameAvg.append(data.tail(1))
            #frameAvgRest.append(dataRest.tail(1))
            #framefpr.append(pfrData.tail(1))
            framefprRest.append(pfrDataRest.tail(1))

    #result = pd.concat(frame)
    #result2 = pd.concat(frameAvg)
    #result3 = pd.concat(framefpr)
    result4 = pd.concat(framefprRest)
    #result5 = pd.concat(framefprRest.tail(1))
    return result4
    #result4 = pd.concat(frameAvgRest)
    #result5 = pd.concat(framefprRest)
    #display(result)
    #result.to_csv(path_analysis+"TrialandRest.csv", index = False)
    #result2.to_csv(path_analysis+"TrialAvg.csv", index = False)
    #result3.to_csv(path_analysis+"CogFreqWarning.csv",index = False)
    #result4.to_csv(path_analysis+"RestAvg.csv", index = False)
    #result5.to_csv(path_analysis+"CogFreqWarning_Rest.csv",index = False)

    print("Done!")

In [ ]:
from datetime import datetime
import time
#Workload
def datamanipulation3(participant):
    ptp = participant

    path_data = "../Experiment Data/" + ptp +"/"
    path_analysis = "../Experiment Analysis/" + ptp + "/"

    responsData = pd.read_csv(path_data + "Responses.csv")
    experimentData = pd.read_csv(path_data + "Classification.csv")
    
    start_time = pd.to_datetime(responsData.WarningStartTime[0:1],format='%H:%M:%S').dt.time
    end_time = pd.to_datetime(responsData.ResponseTime[0:1],format='%H:%M:%S').dt.time

    clock_time = pd.to_datetime(experimentData["System_Time"],format='%H:%M:%S').dt.time

    frame = []
    frameRest = []
    frameAvg = []
    frameAvgRest = []

    for index, row in responsData.iterrows():
        ApplicationName,WarningName,Id,Response,ApplicationId,ParticipantId,ResponseTime,WarningId,WarningStartTime,MoreInfoClicked,MoreInfoTime,Time_Spent= row

        start_time = pd.to_datetime(responsData.WarningStartTime[index:index+1],format='%H:%M:%S').dt.time
        end_time = pd.to_datetime(responsData.ResponseTime[index:index+1],format='%H:%M:%S').dt.time

        #rest time 5sec
        rest_start_time = end_time
        rest_end_time = (pd.to_datetime(responsData.ResponseTime[index:index+1],format='%H:%M:%S') + pd.to_timedelta(5, unit='s')).dt.time

        mask = (clock_time < end_time[index]) & (clock_time >= start_time[index])


        mask2 = (clock_time >= rest_start_time[index]) & (clock_time < rest_end_time[index]) 

        data = experimentData[mask]
        data["AppName"] = ApplicationName
        data["WarnName"] = WarningName
        data["AppId"] = ApplicationId
        data["WarnId"] = WarningId
        data["TimeSpent"] = Time_Spent
        data["PId"] = ParticipantId
        data["Id"] = Id
        count = 0

        dataRest = experimentData[mask2]
        dataRest["WarnName"] = WarningName
        dataRest["Rest"] = index+1
        dataRest["AppName"] = ApplicationName
        dataRest["AppId"] = ApplicationId
        dataRest["WarnId"] = WarningId
        dataRest["TimeSpent"] = "5"
        dataRest["PId"] = ParticipantId

        data.loc['Avg'] = data[['ProbFBDSWorkload','ProbBDSWorkload','ProbAveWorkload',"AppId","WarnId","WarnName","TimeSpent","PId"
                                               ]].mean(axis=0,numeric_only=True).round(2)
        dataRest.loc['AvgRest'] = dataRest[['ProbFBDSWorkload','ProbBDSWorkload','ProbAveWorkload',"AppId","WarnId","WarnName","TimeSpent","PId"
                                               ]].mean(axis=0,numeric_only=True).round(2)


        if not (data.empty):
            frame.append(data)
            frameRest.append(dataRest)
            frameAvg.append(data.tail(1))
            frameAvgRest.append(dataRest.tail(1))

    result = pd.concat(frame)
    result2 = pd.concat(frameAvg)
    result3 = pd.concat(frameRest)
    result4 = pd.concat(frameAvgRest)
    return result,result2,result3,result4
    result.to_csv(path_analysis+"WL.csv", index = False)
    result2.to_csv(path_analysis+"AveWL.csv", index = False)
    result3.to_csv(path_analysis+"WLRest.csv",index = False)
    result4.to_csv(path_analysis+"WLRestAvg.csv", index = False)
    

    print("Done!")

In [ ]:
participants = ["1003_0","1004_0","1005_0","1006_1","1007_2","1008_0","1009_0","1010_0","1011_0","1012_0","1013_0","1015_0","1017_0","1018_0","1019_1","1020_0","1021_0","1022_0","1023_0","1024_0"]
frame = []
frame2 = []
frame3 = []
frame4 = []
for p in participants:
#     result,result2,result3,result4 = datamanipulation3(p)
#     frame.append(result)
#     frame2.append(result2)
#     frame3.append(result3)
#     frame4.append(result4)
    result = datamanipulation2(p)
    #result.loc['Avg'] = result[['AppName','WarnName','TimeSpent','pfrSO','pfrDIS','pfrLENG','pfrHENG','PId']].mean(axis=0,numeric_only=True).round(3)
    frame.append(result)
    #frame2.append(result2)
    print(p + " is done!")
    
result = pd.concat(frame)
#result2 = pd.concat(frame2)
#result4 = pd.concat(frame4)
path_data = "../Experiment Analysis/All Participants/"
#result.to_csv(path_data+"ALL_CogFr.csv", index = False)
result.to_csv(path_data+"ALL_Rest_AVG_1-4_seconds.csv", index = False)


In [ ]:
participants = ["1003_0","1004_0","1005_0","1006_1","1007_2","1008_0","1009_0","1010_0","1011_0","1012_0","1013_0","1015_0","1017_0","1018_0","1019_1","1020_0","1021_0","1022_0","1023_0","1024_0"]
frame = []
for p in participants:
    data = datamanipulation2(p)    
    frame.append(data)
    print(p + " is done!")
    
result = pd.concat(frame)
path_data = "../Experiment Analysis/All Participants/"
result.to_csv(path_data+"All_CogState_average_data_filtered_byState_20p.csv", index = False)
